# Querying in a corpus of documents (of the web, for that matter?)

### The purpose of this class is to understand how a corpus of documents can be indexed in a 'simple' way. After that, we can retrieve the most relevant documents for a given query, or we could also try to perform some clustering.

### To this end, we are going to use a set of sonnets by Shakespeare

In [1]:
# path to the file. You should point this to your data
file = '../data/shakespeare.txt'

# see the document, using the 'head' command
!head -n40 $file

1609

THE SONNETS

by William Shakespeare



                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lus

### Note that if we simply invoke the method `sc.textFile` on the input file, we will read the data line by line. For this exercise we want to create a corpus of documents; hence, we will read the data paragraph by paragraph, so that each paragraph will be like a document.

### The method `newAPIHadoopFile` is quite advanced, so don't worry if you are not familiar with it :)

In [2]:
def myTextFileOpening(file_, sc_, delimiter=' '):
    return sc_.newAPIHadoopFile(file_, "org.apache.hadoop.mapreduce.lib.input.TextInputFormat",
    "org.apache.hadoop.io.LongWritable", "org.apache.hadoop.io.Text",
    conf={"textinputformat.record.delimiter": delimiter}).map(lambda l:l[1])

In [3]:
myTextFileOpening(file, sc, delimiter='\n').take(10)

['1609',
 '',
 'THE SONNETS',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '                     1',
 '  From fairest creatures we desire increase,']

In [4]:
# read data by paragraphs
paragraphs = myTextFileOpening(file, sc, delimiter='\n\n')

paragraphs.take(5)

['1609',
 'THE SONNETS',
 'by William Shakespeare',
 '',
 "                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud buriest thy content,\n  And tender churl mak'st waste in niggarding:\n    Pity the world, or else this glutton be,\n    To eat the world's due, by the grave and thee."]

### One we have our corpus of documents, we have to perform some cleaning on it. For this, we will be using `flatMap` operation form Spark:

![flatMap](https://image.slidesharecdn.com/apachespark-141115145614-conversion-gate01/95/apache-spark-with-scala-28-638.jpg?cb=1416065824)

In [5]:
import re

In [6]:
help(re.sub)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the match object and must return
    a replacement string to be used.



In [7]:
# clean the data: we want to remove spaces (larger than one), empty lines and punctuation
def cleanParagraph(p):
    # convert to lower case
    p = p.lower()
    # remove head and tail spaces, if any
    p = p.strip()
    # remove puntuaction (everything but letters and numbers)
    p = re.sub('[^a-z0-9 ]', '', p)
    # replace multiple spaces by single space
    p = re.sub(' +', ' ', p)
    # remove empty lines from the RDD: for this, we return a list. If the list is empty
    # the flatMap operation will remove it.
    if p!='':
        return [p]
    else:
        return []
test= " Ey cómo te pasas,             tío! 98"
print(cleanParagraph(test))
# output.
# "ey cmo te pasas to 98

['ey cmo te pasas to 98']


In [8]:
# apply the cleanParagraph functoin to paragraphs using flatMap
cleanParagraphs = paragraphs.flatMap(cleanParagraph)

In [9]:
# visualize some of the 'documents' to double check that everything went well
cleanParagraphs.take(5)

['1609',
 'the sonnets',
 'by william shakespeare',
 '1 from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy foe to thy sweet self too cruel thou that art now the worlds fresh ornament and only herald to the gaudy spring within thine own bud buriest thy content and tender churl makst waste in niggarding pity the world or else this glutton be to eat the worlds due by the grave and thee',
 '2 when forty winters shall besiege thy brow and dig deep trenches in thy beautys field thy youths proud livery so gazed on now will be a tattered weed of small worth held then being asked where all thy beauty lies where all the treasure of thy lusty days to say within thine own deep sunken eyes were an alleating shame and thriftless praise how much mor

### For the purpose of querying in the corpus, it's convinient to index the documents with `zipWithIndex`. See the differece with `zipWithUniqueId`.

### Also, we are going to persist the data in main memory, since we may want to perform several queries. The  level of persistance can be changed usgin the `StorageLevel`, for instance `persist(StorageLevel.MEMORY_AND_DISK)`. In order to see the level of persistance you can use the method `getStorageLevel()`.

In [10]:
# add index to 'documents'. 
cleanParagraphsWithIndex = (cleanParagraphs
                            .zipWithIndex() 
                            .map(lambda x: (x[1],x[0])))
# persist the data in RAM 
cleanParagraphsWithIndex.cache()

PythonRDD[9] at RDD at PythonRDD.scala:48

In [11]:
cleanParagraphsWithIndex.take(3)

[(0, '1609'), (1, 'the sonnets'), (2, 'by william shakespeare')]

In [12]:
print(cleanParagraphsWithIndex.getStorageLevel())

Memory Serialized 1x Replicated


In [13]:
from pyspark import StorageLevel
cleanParagraphsWithIndex.unpersist()
cleanParagraphsWithIndex.persist(StorageLevel.MEMORY_ONLY_2)

PythonRDD[9] at RDD at PythonRDD.scala:48

In [14]:
cleanParagraphsWithIndex.take(3)
print(cleanParagraphsWithIndex.getStorageLevel())

Memory Serialized 2x Replicated


In [15]:
print(cleanParagraphsWithIndex.getNumPartitions())
cleanParagraphsWithIndex = cleanParagraphsWithIndex.repartition(10).cache()
print(cleanParagraphsWithIndex.getNumPartitions())

1
10


In [16]:
print(cleanParagraphsWithIndex.getStorageLevel())

Memory Serialized 1x Replicated


In [17]:
# number of documents (that is, paragraphs) in our corpus:
print('There are %d "documents"' % cleanParagraphsWithIndex.count())

There are 6376 "documents"


In [18]:
cleanParagraphsWithIndex.sample()

TypeError: sample() missing 2 required positional arguments: 'withReplacement' and 'fraction'

In [ ]:
from pyspark.mllib.feature import HashingTF, IDF

### Our next task is to *label* each document. One simple approach is to use the well-known  *term frequency–inverse document frequency*, or TF-IDF. Recall what we have seen in the class, and have a look at wikipedia!!!

### In the following, we will be calculating separatedly the TF and IDF terms in a scalable way. Then, we will combine them by *collecting* the IDF RDD (i.e., bringing the data to memory). As I've said, this solution works here, but might fail in a larger corpus...

## 1. TF: term frequency in each document

### Basically, we want a histogram of words for each document in the corpus. In order to leverage the importance of larger documents, we use log-scale instead of the raw frequency of the words.

$$ log(1 + n)$$

In [ ]:
# calculate document frequency:
from math import log

def simpleHist(lista):
    """
    
    :param lista: sequence of words
    :return a historgram of the input lista
    """
    dicc = {}
    for w in lista:
        if w in dicc:
            dicc[w] +=1
        else:
            dicc[w] = 1
    return dicc
print(simpleHist('1 1 1 23 23 hola hola hola 1 a'.split(" ")))    

In [ ]:
def create_hist_for_document(doc):
    '''
    This method accepts a string (doc), and returns its histogram of words 
    '''
    # split input string into array of words
    listOfWords = doc.split(' ')
    # return a dictionary of key = word and value = frequency of that word
    # dicc = {x:listOfWords.count(x) for x in listOfWords}
    dicc = simpleHist(listOfWords)
    # convert to logarithmic scale: this step is optional, we can use directly the frequencies
    dicc = {k: log(v+1) for k,v in dicc.items()}
    return dicc

print(create_hist_for_document('1 1 1 23 23 hola hola hola 1 a'))
# we use assert to check the solution
assert create_hist_for_document('1 1 1 23 23 hola hola hola 1 a') == \
    {'1': log(4+1), 'a': log(1+1), '23': log(2+1), 'hola': log(3+1)}

In [ ]:
cleanParagraphsWithIndex.first()

In [ ]:
# apply the function create_hist_for_document to each 'document' in the RDD
tf = cleanParagraphsWithIndex.map(lambda docs: (docs[0],create_hist_for_document(docs[1])))

In [ ]:
# check everything went well
tf.take(4)

## 2. Inverse document frequency (IDF):

### After having obtained the frequency of words in each document (the so-called TF), we may want to remove terms that are too common. For instance, words like *a*, *and* or *the* are very unlikely to represent a document. Recall from the classroom that such words are known as [*stop-words*](https://en.wikipedia.org/wiki/Stop_words).

### The IDF index of a word is inversely proportional to the occurrance of that word in the corpus and thus, is useful for removing terms that appear frequently in our corpus. Note that we only take into account whether the word appear in a document, not how many times!!!

In [ ]:
from operator import add
from math import log

# get unique words in a document
def word_in_doc(doc):
    '''
    This method return the unique words in a document. Since we want to 
    calculate the occurrance of words among documents, we return a list of 
    tuples (word,1). This structure will be useful later on in the reduce phase.
    '''
    list_words = doc.split(' ')
    unique_words = set(list_words)
    # emit a 1 for each word:
    words_with_ones = [(w,1) for w in unique_words]
    return words_with_ones

# note that we only count 'tal' once (because the sentence passed to words_in_doc represents a single 'document')
assert set(word_in_doc('hola que tal 123 tal')) == set([('tal', 1), ('123', 1), ('que', 1), ('hola', 1)])

    log(1+nunmberOfDocs/float(1+w_f[1])

$$\log(1 + number_of_docs)$$

In [ ]:
# calculate inverse document frequency
def inverse_doc_freq(rdd):
    '''
    This method calculates the inverse document frequency for each term in each document
    Input: and rdd consisting of tuples of (doc_index, doc)
    
    '''
    nunmberOfDocs = rdd.count()
    
    # get an RDD of words with the number of documents this word appears in
    freqWordsInDocs = (rdd.flatMap(lambda docs: word_in_doc(docs[1]))
                       .reduceByKey(add))
    # inverse doc frequency: an RDD of tuples of the form (key=word,value=idf)
    idf = freqWordsInDocs.map(lambda w_f: (w_f[0],log(1+nunmberOfDocs/float(1+w_f[1]))))
    return idf

# Check that everything went well (you can try other sentences, but be sure to change the assert statement accordingly!)
test_corpus = sc.parallelize([(0,'hola que tal 123 tal'),(1,'hola mi amor amor'),(2,'hola que haces amor')])
assert inverse_doc_freq(test_corpus).collectAsMap()  == {'haces': log(1+3.0/(1+1)), 
           'mi': log(1+3.0/(1+1)), '123': log(1+3.0/(1+1)), 'que': log(1+3.0/(1+2)), 
           'tal': log(1+3.0/(1+1)), 'amor': log(1+3.0/(1+2)), 'hola': log(1+3.0/(1+3))}

## 3. TF-IDF

### Now, we have to combine the TF and IDF RDDs. Remember that TF is an RDD of tuples:
###    (index_of_doc, dictionary of word frequencies)
### and IDF is a RDD of tuples: 
###    (word, idf)

### There are several approaches to combine both RDDs. For this excersise, we will follow the simplest approach: collect the IDF RDD as a dictionary. It should be clear that, for a sufficiently large corpus, bringing this data to memory can cause a memoryOutOfBounds exception

In [ ]:
# this might be unsafe!!!
idf = inverse_doc_freq(cleanParagraphsWithIndex).collectAsMap()

In [ ]:
print('Some idf: %s' % [(i,idf[i]) for i in idf][:10])

In [ ]:
# put toghether the tf and the idf
tfidf = tf.mapValues(lambda dicc: {word:tf_*idf[word] for word, tf_ in dicc.items()})
tfidf.cache()

In [ ]:
# Note that until we don't execute this cell, nothing is actually done
tfidf.take(4)
# also, note that if you execute the cell again, the result appears inmediatly, because we persisted the RDD tfidf

** Advanced**: when working on distributed systems, the above will fail. We need to use broadcast variables!!

In [ ]:
idf_br = sc.broadcast(idf)

In [ ]:
idf_br.value

In [ ]:
tf.mapValues(lambda dicc: {k:idf_br.value[k]*tf for k,tf in dicc.items()}).take(10)

## Excercise:

### Combine the TF and IDF RDDs with a join operation. 

### Remember that TF is an RDD of tuples:
###    (index_of_doc, dictionary of word frequencies)
### and IDF is a RDD of tuples: 
###    (word, idf)

### Thus, we have to cast TF as:
###    (word, {index_of_doc: word_frequency})
### Perform the join with idf, so as to get:
###    (word, (idf, {index_of_doc: word_frequency}))
### Multiply the idf and word_frequencies:
###    (word, {index_of_doc: idf*word_frequency})
### And finally return to the requiered structure:
### (index_of_doc, {word: idf*word_frequency})

In [ ]:
tf.first()

In [ ]:
idf_rdd = inverse_doc_freq(cleanParagraphsWithIndex).cache()
idf_rdd.take(10)

In [ ]:
def split_join_tuple(t):
    """
    :params t: tuple of the form (word, ( (doc_id, tf), idf))
    
    """
    word = t[0]
    v = t[1]
    doc_id = v[0][0]
    tf_idf = v[0][1] * v[1]
    return (doc_id, (word, tf_idf))

In [ ]:
tf.flatMap(lambda x: [(w, (x[0],tf_)) for w, tf_ in x[1].items()]).join(idf_rdd).map(split_join_tuple).take(10)

In [ ]:
(tf
 .flatMap(lambda x: [(w, (x[0],tf_)) for w, tf_ in x[1].items()])
 .join(idf_rdd)
 .map(split_join_tuple)
 .groupByKey()
 .mapValues(lambda iter_: dict(list(iter_)))
).take(2)

## 4. Querying

### We are finally in a position to perform our queries on the corpus. The idea is to retrieve the most relevant docuemnts for a given set of words (i.e, the query)

In [ ]:
import numpy as np
def get_paragraphs(tfidf =None, query = None, numberOfDocs = 10):
    return (tfidf
           .map(lambda x: (x[0],sum({x[1][k] if k in x[1] else -1 for k in query}) ))
           .takeOrdered(numberOfDocs,key=lambda x: -x[1]))

def return_docs(query = None, corpus = None, tfidf = None, numberOfDocs =10):
    p = get_paragraphs(tfidf, query, numberOfDocs)
    print('Top-%s paragraphs are %s' % (numberOfDocs,p))
    
    indexes = [i[0] for i in p]
    return corpus.filter(lambda x: x[0] in indexes).collect()

In [ ]:
query = ['william','shakespeare']
return_docs(query=query, corpus=cleanParagraphsWithIndex,tfidf=tfidf,numberOfDocs=5)

### Check out that the word *william* appears several times in the last documents. Thus, we are actually retrieving relevant documents :) 

### However, the last document has more than 20 occurrancies of the word *william*. It seems reasonable that this document should have been returned in a higher position, right? Can you think of an explanation for this?


# 5. Word2Vec

The famous `word2vec` algorithm generates a distributed representation of words. Take a look at the blog: http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/

We could use this technique to find synonimes of a given query, and use them to retrieve the most relevant paragraphs. 

In [ ]:
from pyspark.mllib.feature import Word2Vec

In [ ]:
# Take a look at teh help
Word2Vec??

In [ ]:
# split words in paragraphs
inp = cleanParagraphs.map(lambda p: p.split(' '))

In [ ]:
# set word embedding size
word2vec = Word2Vec().setVectorSize(100).setNumIterations(10)
# take a look at other properties!

In [ ]:
# fit the model
model = word2vec.fit(inp)

In [ ]:
# find synonyms:
synonyms = model.findSynonyms("love",10)

for word, cosine_distance in synonyms:
    print("{}: {}".format(word, cosine_distance))

## 6. Next steps

### - As you might have noticed, we are not considering plurals here. Can you think of a way to account for this?

### - Spark's implementation of TF-IDF is not very robust (actually, it's still marked as *experimental*). The reason for this is that, in order to reduce the complexity of the problem, they use the so called [*hashing trick*](https://en.wikipedia.org/wiki/Feature_hashing). However, as explained in [Spark's documentation](http://spark.apache.org/docs/latest/ml-features.html#tf-idf), such implementation "suffers from potential hash collisions, where different raw features (i.e. words) may become the same term after hashing". 

### - Apart form querying on the corpus, one thing that could be done is to perform some clustering. For this, we could use the tf-idf index to calculate the distance between any pair of documents. Once this is done, we could build the [similarity matrix](https://en.wikipedia.org/wiki/Similarity_measure) and find clusters using, for instance, PCA, k-means, etc. 

### Another approach, more advanced and interesting, is to apply [topic model](https://en.wikipedia.org/wiki/Topic_model) to the corpus. The idea is to find the *latent* topics in the corpus. Every document might then belong to one or more topics (i.e., clusters). One famous algorithm for Topic Model is [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf), and there are implementations of it in Python or R, as well as a [scalable version in Spark](http://spark.apache.org/docs/latest/mllib-clustering.html#latent-dirichlet-allocation-lda) (experimental, so be careful!).